In [ ]:
from pathlib import Path
from typing import Optional

import fenic as fc

config = fc.SessionConfig(
    app_name="json_processing",
    semantic=fc.SemanticConfig(
        language_models={
            "mini": fc.OpenAIModelConfig(
                model_name="gpt-4o-mini",
                rpm=500,
                tpm=200_000
            )
        }
    )
)

# Create session
session = fc.Session.get_or_create(config)


In [ ]:
transcript_path = Path("whisper-transcript.json")

with open(transcript_path, "r") as f:
    json_content = f.read()

# Create dataframe with the JSON string
df = session.create_dataframe([{"json_string": json_content}])

# Cast the JSON string to JSON type
df_json = df.select(
    fc.col("json_string").cast(fc.JsonType).alias("json_data")
)

df_json.show(1)

In [ ]:
# Extract all words from all segments using JQ
# This demonstrates nested array traversal and variable binding in JQ
words_df = df_json.select(
    fc.json.jq(
        fc.col("json_data"),
        # JQ query explanation:
        # - '.segments[] as $seg' iterates through segments, binding each to $seg
        # - '$seg.words[]' iterates through words in each segment
        # - Constructs object with both word-level and segment-level data
        '.segments[] as $seg | $seg.words[] | {word: .word, speaker: .speaker, start: .start, end: .end, probability: .probability, segment_start: $seg.start, segment_end: $seg.end, segment_text: $seg.text}'
    ).alias("word_data")
).explode("word_data")  # Convert array of word objects into separate rows
words_df.show(3)

In [10]:
# Extract scalar values using struct casting and unnest - more efficient than JQ + get_item(0)
# Define schema for word-level data structure
word_schema = fc.StructType([
    fc.StructField("word", fc.StringType),
    fc.StructField("speaker", fc.StringType),
    fc.StructField("start", fc.FloatType),
    fc.StructField("end", fc.FloatType),
    fc.StructField("probability", fc.FloatType),
    fc.StructField("segment_start", fc.FloatType),
    fc.StructField("segment_end", fc.FloatType)
])

# Cast to struct and unnest to automatically extract all fields
words_clean_df = words_df.select(
    fc.col("word_data").cast(word_schema).alias("word_struct")
).unnest("word_struct").select(
    # Rename fields for clarity
    fc.col("word").alias("word_text"),
    fc.col("speaker"),
    fc.col("start").alias("start_time"),
    fc.col("end").alias("end_time"),
    fc.col("probability"),
    fc.col("segment_start"),
    fc.col("segment_end")
)

print("\nScalar extracted fields:")
words_clean_df.show(3)


Scalar extracted fields:
┌───────────┬────────────┬─────────────┬─────────────┬─────────────┬───────────────┬─────────────┐
│ word_text ┆ speaker    ┆ start_time  ┆ end_time    ┆ probability ┆ segment_start ┆ segment_end │
╞═══════════╪════════════╪═════════════╪═════════════╪═════════════╪═══════════════╪═════════════╡
│  Let      ┆ SPEAKER_01 ┆ 2.94        ┆ 3.12        ┆ 0.693848    ┆ 2.94          ┆ 4.48        │
│  me       ┆ SPEAKER_01 ┆ 3.12        ┆ 3.26        ┆ 0.999023    ┆ 2.94          ┆ 4.48        │
│ …         ┆ …          ┆ …           ┆ …           ┆ …           ┆ …             ┆ …           │
│  value.   ┆ SPEAKER_00 ┆ 1486.699951 ┆ 1486.959961 ┆ 1.0         ┆ 1482.060059   ┆ 1486.959961 │
└───────────┴────────────┴─────────────┴─────────────┴─────────────┴───────────────┴─────────────┘


In [11]:
# Add calculated fields - types are already correct from struct schema
# This demonstrates arithmetic operations on struct-extracted data
words_final_df = words_clean_df.select(
    "*",
    # Calculate duration: end_time - start_time (demonstrates arithmetic on struct data)
    (fc.col("end_time") - fc.col("start_time")).alias("duration")
)

print("\n📊 Words DataFrame with calculated duration:")

words_final_df.show(10)


📊 Words DataFrame with calculated duration:
┌────────────┬────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬──────────┐
│ word_text  ┆ speaker    ┆ start_time ┆ end_time   ┆ probabili ┆ segment_s ┆ segment_e ┆ duration │
│            ┆            ┆            ┆            ┆ ty        ┆ tart      ┆ nd        ┆          │
╞════════════╪════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪══════════╡
│  Let       ┆ SPEAKER_01 ┆ 2.94       ┆ 3.12       ┆ 0.693848  ┆ 2.94      ┆ 4.48      ┆ 0.18     │
│  me        ┆ SPEAKER_01 ┆ 3.12       ┆ 3.26       ┆ 0.999023  ┆ 2.94      ┆ 4.48      ┆ 0.14     │
│  ask       ┆ SPEAKER_01 ┆ 3.26       ┆ 3.74       ┆ 0.998047  ┆ 2.94      ┆ 4.48      ┆ 0.48     │
│  you       ┆ SPEAKER_01 ┆ 3.74       ┆ 3.86       ┆ 0.992676  ┆ 2.94      ┆ 4.48      ┆ 0.12     │
│  about     ┆ SPEAKER_01 ┆ 3.86       ┆ 4.1        ┆ 0.999023  ┆ 2.94      ┆ 4.48      ┆ 0.24     │
│ …          ┆ …          ┆ …          ┆ …    

In [12]:
# 2. Create Segments DataFrame (Content-focused)
print("\n📝 Creating Segments DataFrame...")

# Extract segment-level data using JQ
# This demonstrates extracting data at a different granularity level
segments_df = df_json.select(
    fc.json.jq(
        fc.col("json_data"),
        # Extract segment objects with their text, timing, and nested words array
        '.segments[] | {text: .text, start: .start, end: .end, words: .words}'
    ).alias("segment_data")
).explode("segment_data")  # Convert segments array into separate rows

print(f"Extracted {segments_df.count()} segments")
segments_df.show(3)



📝 Creating Segments DataFrame...
Extracted 243 segments
┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ segment_data                                                                                     │
╞══════════════════════════════════════════════════════════════════════════════════════════════════╡
│ {"text":"Let me ask you about AI.","start":2.94,"end":4.48,"words":[{"end":3.12,"probability":0. │
│ 69384765625,"speaker":"SPEAKER_01","start":2.94,"word":"                                         │
│ Let"},{"end":3.26,"probability":0.9990234375,"speaker":"SPEAKER_01","start":3.12,"word":"        │
│ me"},{"end":3.74,"probability":0.998046875,"speaker":"SPEAKER_01","start":3.26,"word":"          │
│ ask"},{"end":3.86,"probability":0.99267578125,"speaker":"SPEAKER_01","start":3.74,"word":"       │
│ you"},{"end":4.1,"probability":0.9990234375,"speaker":"SPEAKER_01","start":3.86,"word":"         │
│ about"},{"end":4.48,"probability

In [13]:
# Extract segment fields using hybrid approach: struct casting + JQ for complex aggregations
# Define schema for basic segment fields (text, start, end)
segment_basic_schema = fc.StructType([
    fc.StructField("text", fc.StringType),
    fc.StructField("start", fc.FloatType),
    fc.StructField("end", fc.FloatType)
])

# First extract basic fields using struct casting, then add complex JQ aggregations
segments_clean_df = segments_df.select(
    # Extract basic segment data using struct casting (more efficient)
    fc.col("segment_data").cast(segment_basic_schema).alias("segment_struct"),
    # Complex array aggregations still use JQ (best tool for this)
    fc.json.jq(fc.col("segment_data"), '.words | length').get_item(0).cast(fc.IntegerType).alias("word_count"),
    fc.json.jq(fc.col("segment_data"), '[.words[].probability] | add / length').get_item(0).cast(fc.FloatType).alias("average_confidence")
).unnest("segment_struct").select(
    # Rename for clarity
    fc.col("text").alias("segment_text"),
    fc.col("start").alias("start_time"),
    fc.col("end").alias("end_time"),
    fc.col("word_count"),
    fc.col("average_confidence")
).select(
    "segment_text",
    "start_time",
    "end_time",
    # Calculate segment duration using DataFrame arithmetic
    (fc.col("end_time") - fc.col("start_time")).alias("duration"),
    "word_count",
    "average_confidence"
)

print("\n📊 Segments DataFrame with calculated metrics:")
segments_clean_df.show(5)


📊 Segments DataFrame with calculated metrics:
┌─────────────────────────┬─────────────┬─────────────┬──────────┬────────────┬────────────────────┐
│ segment_text            ┆ start_time  ┆ end_time    ┆ duration ┆ word_count ┆ average_confidence │
╞═════════════════════════╪═════════════╪═════════════╪══════════╪════════════╪════════════════════╡
│ Let me ask you about    ┆ 2.94        ┆ 4.48        ┆ 1.54     ┆ 6          ┆ 0.941569           │
│ AI.                     ┆             ┆             ┆          ┆            ┆                    │
│ It seems like this year ┆ 4.72        ┆ 11.66       ┆ 6.94     ┆ 22         ┆ 0.981401           │
│ for the entirety of the ┆             ┆             ┆          ┆            ┆                    │
│ human civilization is   ┆             ┆             ┆          ┆            ┆                    │
│ an interesting year for ┆             ┆             ┆          ┆            ┆                    │
│ the development of      ┆             ┆   

In [14]:
# 3. Create Speaker Summary DataFrame (Aggregated)
print("\n🎤 Creating Speaker Summary DataFrame...")

# Use traditional DataFrame aggregations on JSON-extracted data
# This demonstrates hybrid processing: JSON extraction + DataFrame analytics
speaker_summary_df = words_final_df.group_by("speaker").agg(
    fc.count("*").alias("total_words"),                    # Count words per speaker
    fc.avg("probability").alias("average_confidence"),     # Average speech confidence
    fc.min("start_time").alias("first_speaking_time"),     # When speaker first appears
    fc.max("end_time").alias("last_speaking_time"),        # When speaker last appears
    fc.sum("duration").alias("total_speaking_time")        # Total time speaking
).select(
    "speaker",
    "total_words", 
    "total_speaking_time",
    "average_confidence",
    "first_speaking_time",
    "last_speaking_time",
    # Calculate derived metric: words per minute
    (fc.col("total_words") / (fc.col("total_speaking_time") / 60.0)).alias("word_rate")
)

print("\n📊 Speaker Summary DataFrame:")
speaker_summary_df.show()


🎤 Creating Speaker Summary DataFrame...

📊 Speaker Summary DataFrame:
┌────────────┬─────────────┬──────────────┬──────────────┬──────────────┬─────────────┬────────────┐
│ speaker    ┆ total_words ┆ total_speaki ┆ average_conf ┆ first_speaki ┆ last_speaki ┆ word_rate  │
│            ┆             ┆ ng_time      ┆ idence       ┆ ng_time      ┆ ng_time     ┆            │
╞════════════╪═════════════╪══════════════╪══════════════╪══════════════╪═════════════╪════════════╡
│ SPEAKER_00 ┆ 3124        ┆ 912.02002    ┆ 0.969939     ┆ 39.66        ┆ 1486.959961 ┆ 205.521792 │
│ SPEAKER_01 ┆ 1011        ┆ 329.14093    ┆ 0.963907     ┆ 2.94         ┆ 1409.420044 ┆ 184.297943 │
└────────────┴─────────────┴──────────────┴──────────────┴──────────────┴─────────────┴────────────┘


In [15]:
# Summary of what we accomplished
print("\n🎯 JSON Processing Pipeline Summary:")
print("=" * 60)
print("📁 Input: Single JSON file (whisper-transcript.json)\n")
print("📊 Output: 3 structured DataFrames")
print()
print("1. 🔤 Words DataFrame:")
print(f"   - {words_final_df.count()} individual words extracted")
print("   - Fields: word_text, speaker, timing, probability, duration")
print("   - Demonstrates: JQ nested array extraction, type casting")
print()
print("2. 📝 Segments DataFrame:")
print(f"   - {segments_clean_df.count()} conversation segments")
print("   - Fields: text, timing, word_count, average_confidence")
print("   - Demonstrates: JQ aggregations, array operations")
print()
print("3. 🎤 Speaker Summary DataFrame:")
print(f"   - {speaker_summary_df.count()} speakers analyzed")
print("   - Fields: totals, averages, speaking patterns, word rates")
print("   - Demonstrates: DataFrame aggregations on JSON-extracted data")
print()
print("🔧 Key Fenic JSON Features Used:")
print("   ✓ JSON type casting from strings")
print("   ✓ JQ queries for complex nested extraction")
print("   ✓ Array operations and aggregations")
print("   ✓ Type conversion and calculated fields")
print("   ✓ Traditional DataFrame operations on JSON data")

# Clean up
session.stop()


🎯 JSON Processing Pipeline Summary:
📁 Input: Single JSON file (whisper-transcript.json)

📊 Output: 3 structured DataFrames

1. 🔤 Words DataFrame:
   - 4135 individual words extracted
   - Fields: word_text, speaker, timing, probability, duration
   - Demonstrates: JQ nested array extraction, type casting

2. 📝 Segments DataFrame:
   - 243 conversation segments
   - Fields: text, timing, word_count, average_confidence
   - Demonstrates: JQ aggregations, array operations

3. 🎤 Speaker Summary DataFrame:
   - 2 speakers analyzed
   - Fields: totals, averages, speaking patterns, word rates
   - Demonstrates: DataFrame aggregations on JSON-extracted data

🔧 Key Fenic JSON Features Used:
   ✓ JSON type casting from strings
   ✓ JQ queries for complex nested extraction
   ✓ Array operations and aggregations
   ✓ Type conversion and calculated fields
   ✓ Traditional DataFrame operations on JSON data
